<a href="https://colab.research.google.com/github/workingbetter/ITNPBD5_Dissertation/blob/main/draft_6(without_penalty).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms

# Load charging rate data
charging_rate_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_curve_data.xlsx')

# Load charging station location data
station_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_station_location_data.xlsx')

# Convert the charging_rate_df data to numpy arrays for the interpolation function
minutes = charging_rate_df['minutes'].values
soc = charging_rate_df['state_of_charge'].values / 100  # Assuming state of charge is given in percentages

# Assigning given values
initial_soc = 0.4  # initial state of charge
total_distance = 500  # total travel distance in miles
average_speed = 50  # average speed in mph
discharge_rate = 0.004  # discharge rate in SOC per mile
min_soc = 0.09  # minimum SOC

def interpolate_charge_time(start_soc, end_soc):
    start_idx = np.searchsorted(soc, start_soc, side='left')
    if soc[start_idx] > start_soc and start_idx > 0:
        start_idx -= 1

    end_idx = np.searchsorted(soc, end_soc, side='left')
    if soc[end_idx] < end_soc and end_idx < len(soc) - 1:
        end_idx += 1

    if start_idx == end_idx:
        return 0  # Start and end SOC are the same

    start_min = minutes[start_idx]
    end_min = minutes[end_idx]
    start_soc_val = soc[start_idx]
    end_soc_val = soc[end_idx]

    # Linear interpolation formula: y = y1 + ((y2 - y1) / (x2 - x1)) * (x - x1)

    # Time interpolation at the start and end SOC
    start_time = start_min + ((minutes[start_idx+1] - start_min) / (soc[start_idx+1] - start_soc_val)) * (start_soc - start_soc_val)
    end_time = end_min + ((minutes[end_idx-1] - end_min) / (soc[end_idx-1] - end_soc_val)) * (end_soc - end_soc_val)

    return end_time - start_time

# Create the fitness and individual classes
if "FitnessMin" in creator.__dict__:
    del creator.__dict__["FitnessMin"]
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))

creator.create('StationDecision', list)
creator.create('Individual', creator.StationDecision, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('stop_or_not', random.randint, 0, 1)
toolbox.register('charge_to', random.uniform, min_soc, 1)
toolbox.register('station_decision', tools.initCycle, creator.StationDecision, (toolbox.stop_or_not, toolbox.charge_to), n=1)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.station_decision, n=len(station_df))
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    total_time = 0
    soc = initial_soc
    previous_location = 0
    charging_times = []
    charging_stations = []
    socs_before_charging = []
    socs_after_charging = []

    for i, decision in enumerate(individual):
        stop, charge_to = decision

        # Driving time to next station
        distance = station_df.loc[i, 'location(mile)'] - previous_location
        driving_time = (distance / average_speed) * 60  # Convert hours to minutes
        total_time += driving_time

        # Update SOC and location
        soc -= distance * discharge_rate
        previous_location = station_df.loc[i, 'location(mile)']

        if stop:
            # Check if the SOC is enough to reach the station
            if soc < min_soc:
                return float('inf'),  # Return very high fitness value

            # Charging
            # ensure that charge_to is always greater than current soc
            charge_to = max(soc + random.uniform(0, 1 - soc), charge_to)

            # Record SOC before charging
            socs_before_charging.append(soc)

            charging_time = interpolate_charge_time(soc, charge_to)
            total_time += charging_time
            charging_times.append(charging_time)
            charging_stations.append(i)  # Store the station index

            soc = charge_to

            # Record SOC after charging
            socs_after_charging.append(soc)

    # Add the final leg of the journey
    final_distance = total_distance - previous_location
    final_driving_time = (final_distance / average_speed) * 60
    total_time += final_driving_time

    # Check if the SOC is enough to reach the destination
    if soc < min_soc:
        return float('inf'),  # Return very high fitness value

    individual.charging_times = charging_times
    individual.charging_stations = charging_stations
    individual.socs_before_charging = socs_before_charging
    individual.socs_after_charging = socs_after_charging

    return total_time,




# Define the genetic operators
toolbox.register('evaluate', evaluate)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

# Define the main function
def main():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('Avg', np.mean)
    stats.register('Min', np.min)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

    best_individual = hof[0]
    total_travel_time_mins = best_individual.fitness.values[0]
    total_travel_time = f'{int(total_travel_time_mins // 60)} hours {int(total_travel_time_mins % 60)} minutes'

    # Print the total travel time
    print(f'Total travel time: {total_travel_time}')

    # Print charging times and stations
    print('Charging times: ', best_individual.charging_times)
    print('Charging stations: ', best_individual.charging_stations)

    return pop, log, hof

# Run the GA
pop, log, hof = main()

# Extract the best individual
best_individual = hof[0]

# Prepare data for DataFrame
data = {
    'Charging Stations': best_individual.charging_stations,
    'Charging Time(minute)': best_individual.charging_times,
    'SOCs Before Charging': best_individual.socs_before_charging,
    'SOCs After Charging': best_individual.socs_after_charging
}

# Create DataFrame
output_df = pd.DataFrame(data)



# Calculate total travel time
total_travel_time_mins = best_individual.fitness.values[0]
total_travel_time = f'{int(total_travel_time_mins // 60)} hours {int(total_travel_time_mins % 60)} minutes'

# Print the total travel time
print(f'Total travel time: {total_travel_time}')


gen	nevals	Avg    	Min    
0  	100   	1030.49	963.538
1  	64    	1008.32	939.608
2  	57    	993.898	930.806
3  	64    	977.344	930.806
4  	53    	964.741	892.537
5  	64    	953.978	892.537
6  	57    	943.707	878.878
7  	55    	939.346	854.591
8  	64    	936.389	854.591
9  	52    	924.364	854.591
10 	47    	913.569	854.591
11 	59    	910.415	854.591
12 	67    	908.412	854.591
13 	50    	899.454	833.955
14 	56    	896.102	834.125
15 	69    	898.79 	836.819
16 	67    	inf    	828.232
17 	57    	883.584	812.137
18 	52    	875.795	812.137
19 	67    	873.313	812.137
20 	59    	862.291	803.34 
21 	64    	inf    	788.414
22 	69    	851.93 	788.414
23 	67    	848.739	788.414
24 	64    	840.688	788.414
25 	60    	835.991	788.414
26 	63    	837.035	788.414
27 	52    	832.261	788.414
28 	59    	830.369	788.414
29 	49    	828.116	788.414
30 	63    	833.544	788.414
31 	57    	830.458	777.909
32 	62    	827.542	777.909
33 	53    	825.566	777.909
34 	59    	831.244	777.909
35 	58    	831.026	777.909
3

In [ ]:
# Print initial state of charge
print(f"Initial state of charge = {initial_soc}\n")
# Print the total travel time
print(f'Total travel time: {total_travel_time}\n')
# Print the total trip distance
print(f'Total trip distance: {total_distance} miles\n')
# Print travel speed
print(f'Travel speed: {average_speed} mile/hr\n')

# output_df['Distance (mile)'] = output_df['Charging Stations'] * 4

# Print the DataFrame
display(output_df)

Initial state of charge = 0.4

Total travel time: 12 hours 47 minutes

Total trip distance: 500 miles

Travel speed: 50 mile/hr



,Charging Stations,Charging Time(minute),SOCs Before Charging,SOCs After Charging
0,2,31.630208,0.352000,0.913635
1,13,1.599342,0.737635,0.764811
2,27,7.887981,0.540811,0.728571
3,31,4.118531,0.664571,0.752084
4,41,15.094579,0.592084,0.854974
5,45,0.108931,0.790974,0.792063
6,69,2.962200,0.408063,0.477953
7,80,36.520802,0.301953,0.935859
8,82,3.626377,0.903859,0.930123
9,83,5.405681,0.914123,0.948180
